In [1]:
import sys, getopt, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix
from sklearn.preprocessing import MinMaxScaler

from qiskit.circuit.library import PauliFeatureMap, RealAmplitudes, ZZFeatureMap
from qiskit_machine_learning.algorithms import VQC
from qiskit_machine_learning.optimizers import L_BFGS_B

root_folder = 'VQC'
### Globals
# For reproducibility
np.random.seed(42)

# Fixed feature sizes
NUM_FEATURES = 3

# Quantum circuit parameters
IF_PAULI_FEATURE_MAP_LIST = [True, False]
FEATURE_MAP_REPS_LIST = [1, 2, 3, 4, 5]
ANSATZ_REPS_LIST = [1, 2, 3, 4, 5]
ENTANGLEMENT_LIST = ['linear', 'full', 'circular']

# Training hyperparameters
LOSS_FUNCTION = 'cross_entropy'

# Training repetition parameters
N_REPEATS = 10

# Data conf
CLASSIFIER_THRESHOLD = 19

def prepare_dataset_k_fold(X_train, y_train, X_test, y_test):
    """
    Prepare dataset by scaling features.
    """
    # Scale training and test sets separately
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler.fit(np.vstack([X_train, X_test]))  # Fit on combined data for consistency
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, y_train, X_test_scaled, y_test, None, None

def reconfig_quantum_kernel_vqc(if_pauli_feature_map, feature_reps, ansatz_reps, entangle, objective_func_vals):
    """
        Create a quantum kernel
        vqc = VQC(C=20.0, epsilon=0.2, quantum_kernel=kernel)

        Args:
            if_pauli_feature_map: If True, use a Pauli feature map, else ZZ feature map.
            feature_reps: Number of repetitions of quantum circuit for a feature map.
            ansatz_reps: Number of repetitions of quantum circuit for the ansatz.
            entangle: Entanglement type of the feature map.
            objective_func_vals: List to store the objective function values during training.

        Returns:
            qsvr: quantum kernel
    """
    # Define callback function for plotting the training progress
    def callback_graph(weights, obj_func_eval):
        # clear_output(wait=True)
        objective_func_vals.append(obj_func_eval)
        print(f"Iteration: {len(objective_func_vals)}, Objective function value: {obj_func_eval}")
        #plt.rcParams["figure.figsize"] = (12, 6)
        #plt.xlabel("Iteration")
        #plt.ylabel("Objective function value")
        #plt.plot(range(len(objective_func_vals)), objective_func_vals)
        #plt.show()

    # Configure feature map
    if if_pauli_feature_map:
        feature_map = PauliFeatureMap(feature_dimension=NUM_FEATURES, reps=feature_reps, entanglement=entangle)
    else:
        feature_map = ZZFeatureMap(feature_dimension=NUM_FEATURES, reps=feature_reps, entanglement=entangle)

    # Configure ansatz
    ansatz = RealAmplitudes(num_qubits=NUM_FEATURES, reps=ansatz_reps)

    # Configure optimizer
    optimizer = L_BFGS_B(ftol=0.000001, maxiter=20)

    return VQC(feature_map=feature_map,
               ansatz=ansatz,
               optimizer=optimizer,
               callback=callback_graph,
               loss=LOSS_FUNCTION,
               )

def train_vqc(vqc, X_train, y_train, X_test):
    """
        Train based on X_train/y_train (after scaling), return prediction from X_test

        Args:
            vqc: quantum kernel
            X_train:
            y_train
            X_test

        Returns:
    """
    vqc.fit(X_train, y_train)
    return vqc.predict(X_train), vqc.predict(X_test)

def get_arguments(argvs):
    _entangle = ''
    _feature_map_reps = ''
    _ansatz_reps = ''
    _if_pauli_feature = True
    try:
        opts, args = getopt.getopt(argvs, "h:e:f:a:p:", ["entangle=", "feature_map_reps=", "ansatz_reps=", "if_pauli_feature="])
    except getopt.GetoptError:
        print(root_folder + '.py -e <entangle> -f <feature_map_reps> -a <ansatz_reps> -p <if_pauli_feature>')
        sys.exit(2)
    for opt, arg in opts:
        if opt == '-h':
            print(root_folder + '.py -e <entangle> -f <feature_map_reps> -a <ansatz_reps> -p <if_pauli_feature>')
            sys.exit()
        elif opt in ("-e", "--entangle"):
            _entangle = arg
        elif opt in ("-f", "--feature_map_reps"):
            _feature_map_reps = int(arg)
        elif opt in ("-a", "--ansatz_reps"):
            _ansatz_reps = int(arg)
        elif opt in ("-p", "--if_pauli_feature"):
            _if_pauli_feature = eval(arg)
    return _entangle, _feature_map_reps, _ansatz_reps, _if_pauli_feature

if __name__ == "__main__":
    date = '05_31_25_0'
    if not os.path.exists(f'{root_folder}/result'):
        os.makedirs(f'{root_folder}/result')
    if not os.path.exists(f'{root_folder}/logs'):
        os.makedirs(f'{root_folder}/logs')
    try:
        from IPython import get_ipython
        if get_ipython() is not None:
            tmp1, tmp2, tmp3, tmp4 = 'linear', 1, 1, True  # Default values
        else:
            tmp1, tmp2, tmp3, tmp4 = get_arguments(sys.argv[1:])
    except ImportError:
        tmp1, tmp2, tmp3, tmp4 = get_arguments(sys.argv[1:])
    if tmp1 != '':
        ENTANGLEMENT_LIST = [tmp1]
    if tmp2 != '':
        FEATURE_MAP_REPS_LIST = [tmp2]
    if tmp3 != '':
        ANSATZ_REPS_LIST = [tmp3]
    if tmp4 != '':
        IF_PAULI_FEATURE_MAP_LIST = [tmp4]
    print(f"\nFEATURE_MAP_REPS_LIST={FEATURE_MAP_REPS_LIST} "
          f"ANSATZ_REPS_LIST={ANSATZ_REPS_LIST} "
          f"ENTANGLEMENT_LIST={ENTANGLEMENT_LIST} "
          f"IF_PAULI_FEATURE_MAP_LIST={IF_PAULI_FEATURE_MAP_LIST} "
          f"date={date}")
    if len(FEATURE_MAP_REPS_LIST) == 1:
        FEATURE_MAP_REPS_LIST_NAME = FEATURE_MAP_REPS_LIST[0]
    else:
        FEATURE_MAP_REPS_LIST_NAME = FEATURE_MAP_REPS_LIST
    if len(ANSATZ_REPS_LIST) == 1:
        ANSATZ_REPS_LIST_NAME = ANSATZ_REPS_LIST[0]
    else:
        ANSATZ_REPS_LIST_NAME = ANSATZ_REPS_LIST
    if len(ENTANGLEMENT_LIST) == 1:
        ENTANGLEMENT_LIST_NAME = ENTANGLEMENT_LIST[0]
    else:
        ENTANGLEMENT_LIST_NAME = ENTANGLEMENT_LIST
    if len(IF_PAULI_FEATURE_MAP_LIST) == 1:
        IF_PAULI_FEATURE_MAP_LIST_NAME = IF_PAULI_FEATURE_MAP_LIST[0]
    else:
        IF_PAULI_FEATURE_MAP_LIST_NAME = IF_PAULI_FEATURE_MAP_LIST
    IF_PAULI_FEATURE_MAP_LIST_NAME = str(IF_PAULI_FEATURE_MAP_LIST_NAME).replace('False', 'ZZ').replace('True', 'Pauli')
    file_name = f'{root_folder}/result/FMR_{FEATURE_MAP_REPS_LIST_NAME}_AR_{ANSATZ_REPS_LIST_NAME}_E_{ENTANGLEMENT_LIST_NAME}_P_{IF_PAULI_FEATURE_MAP_LIST_NAME}_{date}.csv'
    print('\nTo be saved to', file_name)

    print("\n--- Loading and Preprocessing Data ---")

    train_df = pd.read_csv("Training_Top3Features.csv")
    test_df = pd.read_csv("Testing_Top3Features.csv")
    X_train = train_df.drop('went_on_backorder', axis=1).values
    y_train = train_df['went_on_backorder'].values
    X_test = test_df.drop('went_on_backorder', axis=1).values
    y_test = test_df['went_on_backorder'].values

    print('Training data size: ', X_train.shape[0])
    print('Testing data size: ', X_test.shape[0])

    df = pd.DataFrame(columns=['feature_map_name', 'feature_map_reps', 'ansatz_reps', 'entanglement',
                               'element test', 'actual test', 'predicted test',
                               'element train', 'actual train', 'predicted train',
                               'train_accuracy', 'test_accuracy', 'train_f1', 'test_f1',
                               'train_recall', 'test_recall', 'train_precision', 'test_precision',
                               'train_cm', 'test_cm'])

    print("\n--- Start Training Loop ---")

    for i in range(N_REPEATS):
        X_train_scaled, y_train_scaled, X_test_scaled, y_test_scaled, element_test, element_train = prepare_dataset_k_fold(X_train, y_train, X_test, y_test)
        for if_pauli_feature_map in IF_PAULI_FEATURE_MAP_LIST:
            for feature_map_reps in FEATURE_MAP_REPS_LIST:
                for ansatz_reps in ANSATZ_REPS_LIST:
                    for entanglement in ENTANGLEMENT_LIST:
                        if if_pauli_feature_map:
                            feature_map_name = 'Pauli'
                        else:
                            feature_map_name = 'ZZ'
                        print(f'\n--- Result of training {i+1} ---')
                        print(f'feature_map_name: {feature_map_name} '
                              f'feature_map_reps: {feature_map_reps} '
                              f'ansatz_reps: {ansatz_reps} '
                              f'entanglement: {entanglement} '
                              f'element test: {element_test if element_test is not None else "N/A"}')
                        # conf kernel
                        objective_func_vals = []
                        vqc = reconfig_quantum_kernel_vqc(if_pauli_feature_map=if_pauli_feature_map,
                                                           feature_reps=feature_map_reps,
                                                           ansatz_reps=ansatz_reps,
                                                           entangle=entanglement,
                                                           objective_func_vals=objective_func_vals,
                                                           )

                        # train
                        predict_train, predict_test = train_vqc(vqc, X_train_scaled, y_train_scaled, X_test_scaled)

                        # some conversions
                        all_preds = np.atleast_1d(predict_test)
                        all_targets = np.atleast_1d(y_test_scaled)
                        all_preds_train = np.array(predict_train)
                        all_targets_train = np.array(y_train_scaled)

                        # Calculate metrics
                        train_accuracy = accuracy_score(all_targets_train, all_preds_train)
                        test_accuracy = accuracy_score(all_targets, all_preds)
                        train_f1 = f1_score(all_targets_train, all_preds_train, zero_division=0)
                        test_f1 = f1_score(all_targets, all_preds, zero_division=0)
                        train_recall = recall_score(all_targets_train, all_preds_train, zero_division=0)
                        test_recall = recall_score(all_targets, all_preds, zero_division=0)
                        train_precision = precision_score(all_targets_train, all_preds_train, zero_division=0)
                        test_precision = precision_score(all_targets, all_preds, zero_division=0)
                        train_cm = confusion_matrix(all_targets_train, all_preds_train)
                        test_cm = confusion_matrix(all_targets, all_preds, labels=[0, 1])

                        # Print metrics
                        print(f"Train Accuracy: {train_accuracy:.4f}")
                        print(f"Test Accuracy: {test_accuracy:.4f}")
                        print(f"Train F1 Score: {train_f1:.4f}")
                        print(f"Test F1 Score: {test_f1:.4f}")
                        print(f"Train Recall: {train_recall:.4f}")
                        print(f"Test Recall: {test_recall:.4f}")
                        print(f"Train Precision: {train_precision:.4f}")
                        print(f"Test Precision: {test_precision:.4f}")
                        print(f"Train Confusion Matrix:\n{train_cm}")
                        print(f"Test Confusion Matrix:\n{test_cm}")

                        # save data
                        new_row = {'feature_map_name': feature_map_name,
                                   'feature_map_reps': feature_map_reps,
                                   'ansatz_reps': ansatz_reps,
                                   'entanglement': entanglement,
                                   'element test': element_test if element_test is not None else 'N/A',
                                   'actual test': np.array(all_targets).flatten(),
                                   'predicted test': np.array(all_preds).flatten(),
                                   'element train': element_train if element_train is not None else 'N/A',
                                   'actual train': np.array(all_targets_train).flatten(),
                                   'predicted train': np.array(all_preds_train).flatten(),
                                   'train_accuracy': train_accuracy,
                                   'test_accuracy': test_accuracy,
                                   'train_f1': train_f1,
                                   'test_f1': test_f1,
                                   'train_recall': train_recall,
                                   'test_recall': test_recall,
                                   'train_precision': train_precision,
                                   'test_precision': test_precision,
                                   'train_cm': train_cm.tolist(),
                                   'test_cm': test_cm.tolist()}
                        df.loc[len(df)] = new_row
                        with np.printoptions(linewidth=10000):
                            df.to_csv(file_name, index=False)  # update csv every loop
                        df.at[0, "info"] = [f"DATASET: Training_Top3Features.csv, Testing_Top3Features.csv, LOSS: {LOSS_FUNCTION}"]


FEATURE_MAP_REPS_LIST=[1] ANSATZ_REPS_LIST=[1] ENTANGLEMENT_LIST=['linear'] IF_PAULI_FEATURE_MAP_LIST=[True] date=05_31_25_0

To be saved to VQC/result/FMR_1_AR_1_E_linear_P_Pauli_05_31_25_0.csv

--- Loading and Preprocessing Data ---
Training data size:  10000
Testing data size:  4000

--- Start Training Loop ---

--- Result of training 1 ---
feature_map_name: Pauli feature_map_reps: 1 ansatz_reps: 1 entanglement: linear element test: N/A
Iteration: 1, Objective function value: 1.0217935085742302
Iteration: 2, Objective function value: 0.9993579424357271
Iteration: 3, Objective function value: 0.988629053074912
Iteration: 4, Objective function value: 0.9876665131521584
Iteration: 5, Objective function value: 0.9877041315723847
Iteration: 6, Objective function value: 0.9876548063999951
Iteration: 7, Objective function value: 0.9876893644341206
Iteration: 8, Objective function value: 0.987650395806204
Iteration: 9, Objective function value: 0.98768622157332
Iteration: 10, Objective fun